# CS 830 Final Course Project - Sushma Anand Akoju
## KB2

Knowledge base is as follows:
1. Bat is a mammal.
2. Bat is used in baseball.
3. Bats have a barrel.
4. Bats have wings.
5. Bats can fly.
6. Flying requires wings.
7. Wings are not barrels.
8. Barrels are not wings.
9. Bat can be elegant.
10. There is a brown bat.

11. Query: Elegant Brown Bat has a barrel and can fly.

### LLM generates KB with word sense disambiguation (out of scope of KB) and world knowledge with common sense
#### Following facts are not available in the KB:
- Mammals are animals
- Bat is an animal
- Bat animal does not have a barrel
- Bat animal can fly
- Bat can imply Bat animal
- Bat can imply Bat baseball

#### Skolemized Clausal Normal forms generated by Claude 3.7 Sonnet:

1. ¬Bat_animal(x) ∨ Mammal(x)
2. ¬Bat_baseball(x) ∨ UsedInBaseball(x)
3. ¬Bat_baseball(x) ∨ HasBarrel(x)
4. ¬Bat_animal(x) ∨ HasWings(x)
5. ¬Bat_animal(x) ∨ CanFly(x)
6. ¬CanFly(x) ∨ HasWings(x)
7. ¬HasWings(x) ∨ ¬HasBarrel(x)
8. ¬HasBarrel(x) ∨ ¬HasWings(x)
9. Bat(c1)
10. Elegant(c1)
11. Bat(c2)
12. Brown(c2)
13. ¬Bat(x) ∨ Bat_animal(x) ∨ Bat_baseball(x)
14. ¬Bat_animal(x) ∨ Bat(x)
15. ¬Bat_baseball(x) ∨ Bat(x)

# Query and Negated Query

16. Query: Elegant(x), Brown(x), Bat(x), HasBarrel(x), CanFly(x))

Negated Query (for resolution):
17. ¬Elegant(x) ∨ ¬Brown(x) ∨ ¬Bat(x) ∨ ¬HasBarrel(x) ∨ ¬CanFly(x)

CNF grammar, same as given in assignment:

clause → literal | clause
| literal
literal → predicate
| - predicate
predicate → capitalized-name ( term-list )
term-list → term , term-list
| term
term → capitalized-constant-name
| capitalized-function-name ( term-list )
| lowercase-variable-name


LLM did not follow the CNF grammar. The skolemized clausal normal forms need to be modified. So I had to implement an RR algorithm for two set of CNFs generated by the LLM.

In [24]:
from sympy import symbols, Not, Or, And, to_cnf, simplify_logic, parse_expr, Predicate, ask, Symbol
from sympy.parsing.mathematica import parse_mathematica
from sympy.unify.core import unify, unify_var

In [25]:
raw_clauses = ["¬Bat_animal(x) ∨ Mammal(x)"
"¬Bat_baseball(x) ∨ UsedInBaseball(x)",
"¬Bat_baseball(x) ∨ HasBarrel(x)",
"¬Bat_animal(x) ∨ HasWings(x)",
"¬Bat_animal(x) ∨ CanFly(x)",
"¬CanFly(x) ∨ HasWings(x)",
"¬HasWings(x) ∨ ¬HasBarrel(x)",
"¬HasBarrel(x) ∨ ¬HasWings(x)",
"Bat(c1)",
"Elegant(c1)",
"Bat(c2)",
"Brown(c2)",
"¬Bat(x) ∨ Bat_animal(x) ∨ Bat_baseball(x)",
"¬Bat_animal(x) ∨ Bat(x)",
"¬Bat_baseball(x) ∨ Bat(x)"]

## Parsing using regex and without Sympy

In [26]:
import re
def get_predicate(s):
    if s.startswith('~'):
        return re.findall(r'^\~[A-Za-z0-9]+\(', s)[0][0:-1]
    else:
        return re.findall(r'^[A-Za-z0-9]+\(', s)[0][0:-1]

In [27]:
def get_pred_arguments(s):
    #return skolem constants/functions and variables
    return re.search(r"\((.*)\)", s).group(1)

In [28]:
get_pred_arguments("BrownBat(Brown_bat)"), get_predicate("BrownBat(Brown_bat)")

('Brown_bat', 'BrownBat')

In [29]:
def is_skolem_constant(s):
    flag = s and isinstance(s, str) and (s.isupper() or  s[0].isupper())
    return flag

In [30]:
def isvariable(s):
    return isinstance(s, str) and s.islower() and (s.isalnum() or s.isalpha())

In [31]:
isvariable("x1"), is_skolem_constant("Brown_bat")

(True, True)

In [32]:
# clauses_,variables, skolem_constants, skolem_functions

In [33]:
# parse_expr('~Bat(x) | ~Brown(x) | ~CanFly(x)')
"¬Bat_animal(x) ∨ Mammal(x)"
"¬Bat_baseball(x) ∨ UsedInBaseball(x)",
"¬Bat_baseball(x) ∨ HasBarrel(x)",
"¬Bat_animal(x) ∨ HasWings(x)",
"¬Bat_animal(x) ∨ CanFly(x)",
"¬CanFly(x) ∨ HasWings(x)",
"¬HasWings(x) ∨ ¬HasBarrel(x)",
"¬HasBarrel(x) ∨ ¬HasWings(x)",
"Bat(c1)",
"Elegant(c1)",
"Bat(c2)",
"Brown(c2)",
"¬Bat(x) ∨ Bat_animal(x) ∨ Bat_baseball(x)",
"¬Bat_animal(x) ∨ Bat(x)",
"¬Bat_baseball(x) ∨ Bat(x)"

'¬Bat_baseball(x) ∨ Bat(x)'

 Sympy

### Manually fill in the predicates since CNF grammar is different

In [34]:
Bat_animal1 = Predicate("Bat_animal")
Mammal1 = Predicate("Mammal")
Bat_baseball1 = Predicate("Bat_baseball")
UsedInBaseball1 = Predicate("UsedInBaseball")
HasBarrel1 = Predicate("HasBarrel")
HasWings1 = Predicate("HasWings")
CanFly1 = Predicate("CanFly")
Elegant1 = Predicate("Elegant")
Brown1 = Predicate("Brown")
Bat1 = Predicate("Bat")

x = Symbol('x')
c1 = Symbol('c1')
c2 = Symbol('c2')

Bat = Bat1(c1)
Elegant = Elegant1(c1)
Bat2 = Bat1(c2)
Brown = Brown1(c2)
Bat3 = Bat1(x)

Bat_animal = Bat_animal1(x)
Bat_baseball = Bat_baseball1(x)
Mammal = Mammal1(x)
UsedInBaseball = UsedInBaseball1(x)
HasBarrel = HasBarrel1(x)
HasWings = HasWings1(x)
CanFly = CanFly1(x)

variables = [x]
skolem_constants = [c1, c2]

### Generate KB with clauses using Applied predicates

In [35]:
kb = [(Bat_animal, Mammal),
 (Not(Bat_baseball), UsedInBaseball),
  (Not(Bat_baseball), HasBarrel),
(Not(Bat_animal) , HasWings),
 (Not(Bat_animal) , CanFly),
  (Not(CanFly), HasWings),
(Not(HasWings), Not(HasBarrel)),
 (Not(HasBarrel), Not(HasWings)),
(Bat,),
 (Elegant,),
 (Bat2,),
  (Brown,),
 (Not(Bat3), Bat_animal, Bat_baseball),
(Not(Bat_animal) , Bat3),
 (Not(Bat_baseball) , Bat3)]

predicates = [Bat, Bat_animal, Mammal, Bat_baseball, UsedInBaseball, HasBarrel, HasWings, Not(HasWings), Not(HasBarrel), Not(Bat_baseball),Not(Bat_animal),
       CanFly, Not(CanFly), Elegant, Brown, Bat2]
literals = [Bat1,Bat_animal1, Mammal1, Bat_baseball1, UsedInBaseball1, HasBarrel1, HasWings1, CanFly1,Elegant1, Brown1 ]

neg_query = ( Not(Elegant), Not(Brown), Not(Bat), Not(HasBarrel), Not(CanFly))
kb.append(neg_query)

cnf_clauses = [to_cnf(clause) for clause in kb]

#### Map each predicate with corresponding literal

In [36]:
lit_map = {pred: [] for pred in predicates}
for pred in predicates:
    for lit in literals:
        if pred.has(lit):
            lit_map[pred].append(lit)

In [37]:
lit_map

{Q.Bat(c1): [Q.Bat],
 Q.Bat_animal(x): [Q.Bat_animal],
 Q.Mammal(x): [Q.Mammal],
 Q.Bat_baseball(x): [Q.Bat_baseball],
 Q.UsedInBaseball(x): [Q.UsedInBaseball],
 Q.HasBarrel(x): [Q.HasBarrel],
 Q.HasWings(x): [Q.HasWings],
 ~Q.HasWings(x): [Q.HasWings],
 ~Q.HasBarrel(x): [Q.HasBarrel],
 ~Q.Bat_baseball(x): [Q.Bat_baseball],
 ~Q.Bat_animal(x): [Q.Bat_animal],
 Q.CanFly(x): [Q.CanFly],
 ~Q.CanFly(x): [Q.CanFly],
 Q.Elegant(c1): [Q.Elegant],
 Q.Brown(c2): [Q.Brown],
 Q.Bat(c2): [Q.Bat]}

In [38]:
cnf_clauses

[(Q.Bat_animal(x), Q.Mammal(x)),
 (~Q.Bat_baseball(x), Q.UsedInBaseball(x)),
 (~Q.Bat_baseball(x), Q.HasBarrel(x)),
 (~Q.Bat_animal(x), Q.HasWings(x)),
 (~Q.Bat_animal(x), Q.CanFly(x)),
 (~Q.CanFly(x), Q.HasWings(x)),
 (~Q.HasWings(x), ~Q.HasBarrel(x)),
 (~Q.HasBarrel(x), ~Q.HasWings(x)),
 (Q.Bat(c1),),
 (Q.Elegant(c1),),
 (Q.Bat(c2),),
 (Q.Brown(c2),),
 (~Q.Bat(x), Q.Bat_animal(x), Q.Bat_baseball(x)),
 (~Q.Bat_animal(x), Q.Bat(x)),
 (~Q.Bat_baseball(x), Q.Bat(x)),
 (~Q.Elegant(c1), ~Q.Brown(c2), ~Q.Bat(c1), ~Q.HasBarrel(x), ~Q.CanFly(x))]

### Map predicate to clauses and clause to predicates

In [39]:
cnf_map = {predicate: [] for predicate in predicates}
pred_map = {clause: [] for clause in cnf_clauses}
for predicate in predicates:
    for cnf in cnf_clauses:
        # print(predicate, cnf)
        if cnf.has(predicate):
            cnf_map[predicate].append(cnf)
            pred_map[cnf].append(predicate)
        # else:
            # print(predicate, cnf)

In [40]:
pred_map

{(Q.Bat_animal(x), Q.Mammal(x)): [Q.Bat_animal(x), Q.Mammal(x)],
 (~Q.Bat_baseball(x), Q.UsedInBaseball(x)): [Q.Bat_baseball(x),
  Q.UsedInBaseball(x),
  ~Q.Bat_baseball(x)],
 (~Q.Bat_baseball(x), Q.HasBarrel(x)): [Q.Bat_baseball(x),
  Q.HasBarrel(x),
  ~Q.Bat_baseball(x)],
 (~Q.Bat_animal(x), Q.HasWings(x)): [Q.Bat_animal(x),
  Q.HasWings(x),
  ~Q.Bat_animal(x)],
 (~Q.Bat_animal(x), Q.CanFly(x)): [Q.Bat_animal(x),
  ~Q.Bat_animal(x),
  Q.CanFly(x)],
 (~Q.CanFly(x), Q.HasWings(x)): [Q.HasWings(x), Q.CanFly(x), ~Q.CanFly(x)],
 (~Q.HasWings(x), ~Q.HasBarrel(x)): [Q.HasBarrel(x),
  Q.HasWings(x),
  ~Q.HasWings(x),
  ~Q.HasBarrel(x)],
 (~Q.HasBarrel(x), ~Q.HasWings(x)): [Q.HasBarrel(x),
  Q.HasWings(x),
  ~Q.HasWings(x),
  ~Q.HasBarrel(x)],
 (Q.Bat(c1),): [Q.Bat(c1)],
 (Q.Elegant(c1),): [Q.Elegant(c1)],
 (Q.Bat(c2),): [Q.Bat(c2)],
 (Q.Brown(c2),): [Q.Brown(c2)],
 (~Q.Bat(x), Q.Bat_animal(x), Q.Bat_baseball(x)): [Q.Bat_animal(x),
  Q.Bat_baseball(x)],
 (~Q.Bat_animal(x), Q.Bat(x)): [Q.Bat_a

In [41]:
for k,v in cnf_map.items():
    if Not(k) in v:
        print(k, v)

### Map literals to clauses

In [42]:
cnf_lit_map = {literal: [] for literal in literals}
for literal in literals:
    for cnf in cnf_clauses:
        # print(predicate, cnf)
        if cnf.has(literal):
            cnf_lit_map[literal].append(cnf)
        # else:
            # print(predicate, cnf)


In [43]:
cnf_lit_map

{Q.Bat: [(Q.Bat(c1),),
  (Q.Bat(c2),),
  (~Q.Bat(x), Q.Bat_animal(x), Q.Bat_baseball(x)),
  (~Q.Bat_animal(x), Q.Bat(x)),
  (~Q.Bat_baseball(x), Q.Bat(x)),
  (~Q.Elegant(c1), ~Q.Brown(c2), ~Q.Bat(c1), ~Q.HasBarrel(x), ~Q.CanFly(x))],
 Q.Bat_animal: [(Q.Bat_animal(x), Q.Mammal(x)),
  (~Q.Bat_animal(x), Q.HasWings(x)),
  (~Q.Bat_animal(x), Q.CanFly(x)),
  (~Q.Bat(x), Q.Bat_animal(x), Q.Bat_baseball(x)),
  (~Q.Bat_animal(x), Q.Bat(x))],
 Q.Mammal: [(Q.Bat_animal(x), Q.Mammal(x))],
 Q.Bat_baseball: [(~Q.Bat_baseball(x), Q.UsedInBaseball(x)),
  (~Q.Bat_baseball(x), Q.HasBarrel(x)),
  (~Q.Bat(x), Q.Bat_animal(x), Q.Bat_baseball(x)),
  (~Q.Bat_baseball(x), Q.Bat(x))],
 Q.UsedInBaseball: [(~Q.Bat_baseball(x), Q.UsedInBaseball(x))],
 Q.HasBarrel: [(~Q.Bat_baseball(x), Q.HasBarrel(x)),
  (~Q.HasWings(x), ~Q.HasBarrel(x)),
  (~Q.HasBarrel(x), ~Q.HasWings(x)),
  (~Q.Elegant(c1), ~Q.Brown(c2), ~Q.Bat(c1), ~Q.HasBarrel(x), ~Q.CanFly(x))],
 Q.HasWings: [(~Q.Bat_animal(x), Q.HasWings(x)),
  (~Q.CanFly

In [44]:
cl = cnf_clauses[10]
cl.atoms(), cl.args, cnf_clauses[10].is_Not, cnf_clauses[9].is_Not, isinstance(cnf_clauses[0], Or), isinstance(cnf_clauses[10], Or), cnf_clauses[0].args[0].has(x)

({Bat, c2}, (Q.Bat(c2),), False, False, False, False, True)

In [45]:
cnf_clauses[10].has(Not(CanFly)), cnf_clauses[10].has(cnf_clauses[10]), cnf_clauses[10].has(cnf_clauses[10].args[0]), cnf_clauses[0].has(cnf_clauses[0])

(False, True, True, True)

### My Solution using Sympy without using Or but instead make clause as a tuple like in the assignment

In [46]:
predicates

[Q.Bat(c1),
 Q.Bat_animal(x),
 Q.Mammal(x),
 Q.Bat_baseball(x),
 Q.UsedInBaseball(x),
 Q.HasBarrel(x),
 Q.HasWings(x),
 ~Q.HasWings(x),
 ~Q.HasBarrel(x),
 ~Q.Bat_baseball(x),
 ~Q.Bat_animal(x),
 Q.CanFly(x),
 ~Q.CanFly(x),
 Q.Elegant(c1),
 Q.Brown(c2),
 Q.Bat(c2)]

#### Map each predicate with corresponding literal

In [47]:
lit_map = {pred: [] for pred in predicates}
for pred in predicates:
    for lit in literals:
        if pred.has(lit):
            lit_map[pred].append(lit)

In [48]:
lit_map

{Q.Bat(c1): [Q.Bat],
 Q.Bat_animal(x): [Q.Bat_animal],
 Q.Mammal(x): [Q.Mammal],
 Q.Bat_baseball(x): [Q.Bat_baseball],
 Q.UsedInBaseball(x): [Q.UsedInBaseball],
 Q.HasBarrel(x): [Q.HasBarrel],
 Q.HasWings(x): [Q.HasWings],
 ~Q.HasWings(x): [Q.HasWings],
 ~Q.HasBarrel(x): [Q.HasBarrel],
 ~Q.Bat_baseball(x): [Q.Bat_baseball],
 ~Q.Bat_animal(x): [Q.Bat_animal],
 Q.CanFly(x): [Q.CanFly],
 ~Q.CanFly(x): [Q.CanFly],
 Q.Elegant(c1): [Q.Elegant],
 Q.Brown(c2): [Q.Brown],
 Q.Bat(c2): [Q.Bat]}

### kb_clauses

In [49]:
kb

[(Q.Bat_animal(x), Q.Mammal(x)),
 (~Q.Bat_baseball(x), Q.UsedInBaseball(x)),
 (~Q.Bat_baseball(x), Q.HasBarrel(x)),
 (~Q.Bat_animal(x), Q.HasWings(x)),
 (~Q.Bat_animal(x), Q.CanFly(x)),
 (~Q.CanFly(x), Q.HasWings(x)),
 (~Q.HasWings(x), ~Q.HasBarrel(x)),
 (~Q.HasBarrel(x), ~Q.HasWings(x)),
 (Q.Bat(c1),),
 (Q.Elegant(c1),),
 (Q.Bat(c2),),
 (Q.Brown(c2),),
 (~Q.Bat(x), Q.Bat_animal(x), Q.Bat_baseball(x)),
 (~Q.Bat_animal(x), Q.Bat(x)),
 (~Q.Bat_baseball(x), Q.Bat(x)),
 (~Q.Elegant(c1), ~Q.Brown(c2), ~Q.Bat(c1), ~Q.HasBarrel(x), ~Q.CanFly(x))]

### Map clauses to predicates

In [50]:
kb_map = {predicate: [] for predicate in predicates}
pred_map = {clause: [] for clause in kb}
for predicate in predicates:
    for cnf in kb:
        # print(predicate, cnf)
        if predicate in cnf:
            # print("True")
            kb_map[predicate].append(cnf)
            pred_map[cnf].append(predicate)
        # else:
            # print(predicate, cnf)

### Map clauses to predicate, argument pairs of each clause as a list
### Map predicates with arguments

In [51]:
kb_map = {predicate: {} for predicate in predicates}
pred_args = {predicate: {} for predicate in predicates}
for predicate in predicates:
    for cnf in kb:
        # print(predicate, cnf)
        if predicate in cnf:
            # print("True")
            this_cnf = []
            for lit in cnf:
                # if lit not in kb_map[predicate]:
                #     kb_map[predicate][lit] = []
                if isinstance(lit, Not) or lit.is_Not:
                    pred = lit.args[0].args[0]
                    arg = lit.args[0].args[1]
                    this_cnf.append([pred, arg])
                    if lit not in pred_args[predicate]:
                        pred_args[predicate][lit] = []
                    if [pred, arg] not in pred_args[predicate][lit]:
                        pred_args[predicate][lit].append([pred, arg])

                else:
                    pred = lit.args[0]
                    arg = lit.args[1]
                    this_cnf.append([pred, arg])
                    if lit not in pred_args[predicate]:
                        pred_args[predicate][lit] = []
                    if [pred, arg] not in pred_args[predicate][lit]:
                        pred_args[predicate][lit].append([pred, arg])
                # print(this_cnf, lit, kb_map[predicate],lit in kb_map[predicate])
                kb_map[predicate][cnf] = this_cnf



In [52]:
kb_map

{Q.Bat(c1): {(Q.Bat(c1),): [[Q.Bat, c1]]},
 Q.Bat_animal(x): {(Q.Bat_animal(x), Q.Mammal(x)): [[Q.Bat_animal, x],
   [Q.Mammal, x]],
  (~Q.Bat(x), Q.Bat_animal(x), Q.Bat_baseball(x)): [[Q.Bat, x],
   [Q.Bat_animal, x],
   [Q.Bat_baseball, x]]},
 Q.Mammal(x): {(Q.Bat_animal(x), Q.Mammal(x)): [[Q.Bat_animal, x],
   [Q.Mammal, x]]},
 Q.Bat_baseball(x): {(~Q.Bat(x),
   Q.Bat_animal(x),
   Q.Bat_baseball(x)): [[Q.Bat, x], [Q.Bat_animal, x], [Q.Bat_baseball, x]]},
 Q.UsedInBaseball(x): {(~Q.Bat_baseball(x),
   Q.UsedInBaseball(x)): [[Q.Bat_baseball, x], [Q.UsedInBaseball, x]]},
 Q.HasBarrel(x): {(~Q.Bat_baseball(x), Q.HasBarrel(x)): [[Q.Bat_baseball, x],
   [Q.HasBarrel, x]]},
 Q.HasWings(x): {(~Q.Bat_animal(x), Q.HasWings(x)): [[Q.Bat_animal, x],
   [Q.HasWings, x]],
  (~Q.CanFly(x), Q.HasWings(x)): [[Q.CanFly, x], [Q.HasWings, x]]},
 ~Q.HasWings(x): {(~Q.HasWings(x), ~Q.HasBarrel(x)): [[Q.HasWings, x],
   [Q.HasBarrel, x]],
  (~Q.HasBarrel(x), ~Q.HasWings(x)): [[Q.HasBarrel, x], [Q.HasWing

In [53]:
pred_map

{(Q.Bat_animal(x), Q.Mammal(x)): [Q.Bat_animal(x), Q.Mammal(x)],
 (~Q.Bat_baseball(x), Q.UsedInBaseball(x)): [Q.UsedInBaseball(x),
  ~Q.Bat_baseball(x)],
 (~Q.Bat_baseball(x), Q.HasBarrel(x)): [Q.HasBarrel(x), ~Q.Bat_baseball(x)],
 (~Q.Bat_animal(x), Q.HasWings(x)): [Q.HasWings(x), ~Q.Bat_animal(x)],
 (~Q.Bat_animal(x), Q.CanFly(x)): [~Q.Bat_animal(x), Q.CanFly(x)],
 (~Q.CanFly(x), Q.HasWings(x)): [Q.HasWings(x), ~Q.CanFly(x)],
 (~Q.HasWings(x), ~Q.HasBarrel(x)): [~Q.HasWings(x), ~Q.HasBarrel(x)],
 (~Q.HasBarrel(x), ~Q.HasWings(x)): [~Q.HasWings(x), ~Q.HasBarrel(x)],
 (Q.Bat(c1),): [Q.Bat(c1)],
 (Q.Elegant(c1),): [Q.Elegant(c1)],
 (Q.Bat(c2),): [Q.Bat(c2)],
 (Q.Brown(c2),): [Q.Brown(c2)],
 (~Q.Bat(x), Q.Bat_animal(x), Q.Bat_baseball(x)): [Q.Bat_animal(x),
  Q.Bat_baseball(x)],
 (~Q.Bat_animal(x), Q.Bat(x)): [~Q.Bat_animal(x)],
 (~Q.Bat_baseball(x), Q.Bat(x)): [~Q.Bat_baseball(x)],
 (~Q.Elegant(c1),
  ~Q.Brown(c2),
  ~Q.Bat(c1),
  ~Q.HasBarrel(x),
  ~Q.CanFly(x)): [~Q.HasBarrel(x), ~Q.C

In [54]:
pred_args

{Q.Bat(c1): {Q.Bat(c1): [[Q.Bat, c1]]},
 Q.Bat_animal(x): {Q.Bat_animal(x): [[Q.Bat_animal, x]],
  Q.Mammal(x): [[Q.Mammal, x]],
  ~Q.Bat(x): [[Q.Bat, x]],
  Q.Bat_baseball(x): [[Q.Bat_baseball, x]]},
 Q.Mammal(x): {Q.Bat_animal(x): [[Q.Bat_animal, x]],
  Q.Mammal(x): [[Q.Mammal, x]]},
 Q.Bat_baseball(x): {~Q.Bat(x): [[Q.Bat, x]],
  Q.Bat_animal(x): [[Q.Bat_animal, x]],
  Q.Bat_baseball(x): [[Q.Bat_baseball, x]]},
 Q.UsedInBaseball(x): {~Q.Bat_baseball(x): [[Q.Bat_baseball, x]],
  Q.UsedInBaseball(x): [[Q.UsedInBaseball, x]]},
 Q.HasBarrel(x): {~Q.Bat_baseball(x): [[Q.Bat_baseball, x]],
  Q.HasBarrel(x): [[Q.HasBarrel, x]]},
 Q.HasWings(x): {~Q.Bat_animal(x): [[Q.Bat_animal, x]],
  Q.HasWings(x): [[Q.HasWings, x]],
  ~Q.CanFly(x): [[Q.CanFly, x]]},
 ~Q.HasWings(x): {~Q.HasWings(x): [[Q.HasWings, x]],
  ~Q.HasBarrel(x): [[Q.HasBarrel, x]]},
 ~Q.HasBarrel(x): {~Q.HasWings(x): [[Q.HasWings, x]],
  ~Q.HasBarrel(x): [[Q.HasBarrel, x]],
  ~Q.Elegant(c1): [[Q.Elegant, c1]],
  ~Q.Brown(c2): [[

In [55]:
Not(Bat1(x)).args[0].args

(Q.Bat, x)

In [56]:
for k,v in kb_map.items():
    if Not(k) in v:
        print(k, v)

### Map literals to clauses

In [57]:
cnf_lit_map = {literal: [] for literal in literals}
for literal in literals:
    for cnf in kb:
        # print(literal, str(cnf[0]))
        for lit in cnf:
            if str(literal) in str(lit):
                cnf_lit_map[literal].append(cnf)
        # else:
            # print(predicate, cnf)


In [58]:
cnf_lit_map

{Q.Bat: [(Q.Bat_animal(x), Q.Mammal(x)),
  (~Q.Bat_baseball(x), Q.UsedInBaseball(x)),
  (~Q.Bat_baseball(x), Q.HasBarrel(x)),
  (~Q.Bat_animal(x), Q.HasWings(x)),
  (~Q.Bat_animal(x), Q.CanFly(x)),
  (Q.Bat(c1),),
  (Q.Bat(c2),),
  (~Q.Bat(x), Q.Bat_animal(x), Q.Bat_baseball(x)),
  (~Q.Bat(x), Q.Bat_animal(x), Q.Bat_baseball(x)),
  (~Q.Bat(x), Q.Bat_animal(x), Q.Bat_baseball(x)),
  (~Q.Bat_animal(x), Q.Bat(x)),
  (~Q.Bat_animal(x), Q.Bat(x)),
  (~Q.Bat_baseball(x), Q.Bat(x)),
  (~Q.Bat_baseball(x), Q.Bat(x)),
  (~Q.Elegant(c1), ~Q.Brown(c2), ~Q.Bat(c1), ~Q.HasBarrel(x), ~Q.CanFly(x))],
 Q.Bat_animal: [(Q.Bat_animal(x), Q.Mammal(x)),
  (~Q.Bat_animal(x), Q.HasWings(x)),
  (~Q.Bat_animal(x), Q.CanFly(x)),
  (~Q.Bat(x), Q.Bat_animal(x), Q.Bat_baseball(x)),
  (~Q.Bat_animal(x), Q.Bat(x))],
 Q.Mammal: [(Q.Bat_animal(x), Q.Mammal(x))],
 Q.Bat_baseball: [(~Q.Bat_baseball(x), Q.UsedInBaseball(x)),
  (~Q.Bat_baseball(x), Q.HasBarrel(x)),
  (~Q.Bat(x), Q.Bat_animal(x), Q.Bat_baseball(x)),
  (~Q.

### Since constants, variables were already extracted during parsing, check if a given input has a variable, has a constant or is a predicate or variable or constant.

In [59]:
#helper fucntions.

def has_variable(literal, variable):
    return literal.has(variable)

def has_skolem_constant(literal, skolem_constant):
    return literal.has(skolem_constant)

def is_variable(variable, variables):
    return True if variable in variables else False

def is_skolem_constant(constant, skolem_constants):
    return True if constant in skolem_constants else False

def is_predicate(predicate, predicates):
    return True if predicate in predicates else False



### Looking for complementary predicates of Sympy
#### Sympy's Applied predicates have arguments that do not retain the Not, Or conditionals.
#### Check each part of clause for itself: predicate, argument only after checking if topmost node in the tree is a Not operator and vice versa.

In [60]:
def is_complement(lit1, lit2):
    if (isinstance(lit1, Not) or lit1.is_Not) and not (isinstance(lit2, Not) or lit2.is_Not):
        pred1 = lit1.args[0].args[0] #first returns Not(this_lit), access this_lit
        arg1 = lit1.args[0].args[1]
        pred2 = lit2.args[0] #first returns Not(this_lit), access this_lit
        arg2 = lit2.args[1]
        if pred2.has(pred1): #and arg1.has(arg2):
            return True
    elif ( isinstance(lit2, Not) or lit2.is_Not) and not (isinstance(lit1, Not) or lit1.is_Not):
        pred1 = lit1.args[0] #first returns Not(this_lit), access this_lit
        arg1 = lit1.args[1]
        pred2 = lit2.args[0].args[0] #first returns Not(this_lit), access this_lit
        arg2 = lit2.args[0].args[1]
        if pred2.has(pred1): #and arg1.has(arg2):
            return True
    return False

def is_negation(clause1, clause2, pred_map):
    for lit1 in pred_map[clause1]:
        for lit2 in pred_map[clause2]:
            if is_complement(lit1, lit2):
                return True
    return False

In [61]:
# Bat1(Brown_bat).has(Not(Not(Bat1(x)).args[0].args[0])), Bat1(Brown_bat).has(Not(Bat1(x)).args[0].args[0]), Bat1(Brown_bat).has(Not(Bat1(x)).args[0].args[0]),Not(Not(Bat1(x)).args[0].args[0])

In [62]:
is_negation(kb[0], kb[9], pred_map),kb[0][0], kb[9][0] #different arguments, complementary pair of predicates

(False, Q.Bat_animal(x), Q.Elegant(c1))

In [63]:
kb[10][0].args[0].args

(Bat,)

In [65]:
is_complement(Bat1(c2), Not(Bat1(x))), is_complement(Not(Bat1(c1)), Not(Bat1(x))), is_complement(Not(Bat1(c1)), Bat1(x)), is_complement(Bat1(c1), Bat1(x))

(True, False, True, False)

In [66]:
clauses_map = {}
for k,clause in kb_map.items():
    for k1,v in clause.items():
        clauses_map[k1] = v

In [67]:
clauses_map

{(Q.Bat(c1),): [[Q.Bat, c1]],
 (Q.Bat_animal(x), Q.Mammal(x)): [[Q.Bat_animal, x], [Q.Mammal, x]],
 (~Q.Bat(x), Q.Bat_animal(x), Q.Bat_baseball(x)): [[Q.Bat, x],
  [Q.Bat_animal, x],
  [Q.Bat_baseball, x]],
 (~Q.Bat_baseball(x), Q.UsedInBaseball(x)): [[Q.Bat_baseball, x],
  [Q.UsedInBaseball, x]],
 (~Q.Bat_baseball(x), Q.HasBarrel(x)): [[Q.Bat_baseball, x], [Q.HasBarrel, x]],
 (~Q.Bat_animal(x), Q.HasWings(x)): [[Q.Bat_animal, x], [Q.HasWings, x]],
 (~Q.CanFly(x), Q.HasWings(x)): [[Q.CanFly, x], [Q.HasWings, x]],
 (~Q.HasWings(x), ~Q.HasBarrel(x)): [[Q.HasWings, x], [Q.HasBarrel, x]],
 (~Q.HasBarrel(x), ~Q.HasWings(x)): [[Q.HasBarrel, x], [Q.HasWings, x]],
 (~Q.Elegant(c1),
  ~Q.Brown(c2),
  ~Q.Bat(c1),
  ~Q.HasBarrel(x),
  ~Q.CanFly(x)): [[Q.Elegant, c1], [Q.Brown, c2], [Q.Bat, c1], [Q.HasBarrel,
   x], [Q.CanFly, x]],
 (~Q.Bat_baseball(x), Q.Bat(x)): [[Q.Bat_baseball, x], [Q.Bat, x]],
 (~Q.Bat_animal(x), Q.CanFly(x)): [[Q.Bat_animal, x], [Q.CanFly, x]],
 (~Q.Bat_animal(x), Q.Bat(x)):

In [68]:
clauses = clauses_map.keys()
allclauses = set(clauses)
newclauses = set(clauses)
negated = []
for c1, v1 in clauses_map.items():
    for c2, v2 in clauses_map.items():
        if c1 != c2:
            print(c1,c2,is_negation(c1, c2, pred_map))
            if is_negation(c1, c2, pred_map):
                if [v1,v2] not in negated:
                    negated.append([v1, v2])

(Q.Bat(c1),) (Q.Bat_animal(x), Q.Mammal(x)) False
(Q.Bat(c1),) (~Q.Bat(x), Q.Bat_animal(x), Q.Bat_baseball(x)) False
(Q.Bat(c1),) (~Q.Bat_baseball(x), Q.UsedInBaseball(x)) False
(Q.Bat(c1),) (~Q.Bat_baseball(x), Q.HasBarrel(x)) False
(Q.Bat(c1),) (~Q.Bat_animal(x), Q.HasWings(x)) False
(Q.Bat(c1),) (~Q.CanFly(x), Q.HasWings(x)) False
(Q.Bat(c1),) (~Q.HasWings(x), ~Q.HasBarrel(x)) False
(Q.Bat(c1),) (~Q.HasBarrel(x), ~Q.HasWings(x)) False
(Q.Bat(c1),) (~Q.Elegant(c1), ~Q.Brown(c2), ~Q.Bat(c1), ~Q.HasBarrel(x), ~Q.CanFly(x)) False
(Q.Bat(c1),) (~Q.Bat_baseball(x), Q.Bat(x)) False
(Q.Bat(c1),) (~Q.Bat_animal(x), Q.CanFly(x)) False
(Q.Bat(c1),) (~Q.Bat_animal(x), Q.Bat(x)) False
(Q.Bat(c1),) (Q.Elegant(c1),) False
(Q.Bat(c1),) (Q.Brown(c2),) False
(Q.Bat(c1),) (Q.Bat(c2),) False
(Q.Bat_animal(x), Q.Mammal(x)) (Q.Bat(c1),) False
(Q.Bat_animal(x), Q.Mammal(x)) (~Q.Bat(x), Q.Bat_animal(x), Q.Bat_baseball(x)) False
(Q.Bat_animal(x), Q.Mammal(x)) (~Q.Bat_baseball(x), Q.UsedInBaseball(x)) False


In [69]:
negated

[[[[Q.Bat_animal, x], [Q.Mammal, x]], [[Q.Bat_animal, x], [Q.HasWings, x]]],
 [[[Q.Bat_animal, x], [Q.Mammal, x]], [[Q.Bat_animal, x], [Q.CanFly, x]]],
 [[[Q.Bat_animal, x], [Q.Mammal, x]], [[Q.Bat_animal, x], [Q.Bat, x]]],
 [[[Q.Bat, x], [Q.Bat_animal, x], [Q.Bat_baseball, x]],
  [[Q.Bat_baseball, x], [Q.UsedInBaseball, x]]],
 [[[Q.Bat, x], [Q.Bat_animal, x], [Q.Bat_baseball, x]],
  [[Q.Bat_baseball, x], [Q.HasBarrel, x]]],
 [[[Q.Bat, x], [Q.Bat_animal, x], [Q.Bat_baseball, x]],
  [[Q.Bat_animal, x], [Q.HasWings, x]]],
 [[[Q.Bat, x], [Q.Bat_animal, x], [Q.Bat_baseball, x]],
  [[Q.Bat_baseball, x], [Q.Bat, x]]],
 [[[Q.Bat, x], [Q.Bat_animal, x], [Q.Bat_baseball, x]],
  [[Q.Bat_animal, x], [Q.CanFly, x]]],
 [[[Q.Bat, x], [Q.Bat_animal, x], [Q.Bat_baseball, x]],
  [[Q.Bat_animal, x], [Q.Bat, x]]],
 [[[Q.Bat_baseball, x], [Q.UsedInBaseball, x]],
  [[Q.Bat, x], [Q.Bat_animal, x], [Q.Bat_baseball, x]]],
 [[[Q.Bat_baseball, x], [Q.HasBarrel, x]],
  [[Q.Bat, x], [Q.Bat_animal, x], [Q.Bat_base

In [70]:
variables, skolem_constants

([x], [c1, c2])

### Unify and Substitute implementation

In [71]:
def update_pred_map(new_clause, pred_map):
    if new_clause not in pred_map:
        pred_map[new_clause] = []
    for lit in new_clause:
        pred_map[new_clause].append(lit)
    return pred_map

def update_clauses_map(new_clause, clauses_map):

    this_cnf = []
    for lit in new_clause:
        # if lit not in kb_map[predicate]:
        #     kb_map[predicate][lit] = []
        if isinstance(lit, Not) or lit.is_Not:
            pred = lit.args[0].args[0]
            arg = lit.args[0].args[1]
            this_cnf.append([pred, arg])
        else:
            pred = lit.args[0]
            arg = lit.args[1]
            this_cnf.append([pred, arg])
            if lit not in pred_args[predicate]:
                pred_args[predicate][lit] = []
            if [pred, arg] not in pred_args[predicate][lit]:
                pred_args[predicate][lit].append([pred, arg])
        # print(this_cnf, lit, kb_map[predicate],lit in kb_map[predicate])
    clauses_map[new_clause] = this_cnf
    return clauses_map



def unify(x, y, s={}):
    # print("start unify",x,y,s)
    if isinstance(s, bool) and s is False:
        # print("isinstance(s, bool) False", x,y,s)
        return False
    if x == y:
        # print("x=y", x,y)
        return s
    if x in variables or x in skolem_constants:
        # print(" unify_variable x",x)
        return unify_variable(x, y, s)
    if is_variable(y, variables) or is_skolem_constant(y, skolem_constants):
        # print("unify_variable y",y)
        return unify_variable(y, x, s)
    if not isinstance(x, list) or not isinstance(y, list):
        # print(x,y)
        return False
    if len(x) != len(y):
        # print("length", x,y)
        return False
    return unify(x[1:], y[1:], unify(x[0],y[0], s))

def unify_variable(var, x, s):
    # print("unify_variable", var, x, s)
    if var in s:
        return unify(s[var], x, s)
    if x in s:
        return unify(var, s[x], s)
    if occurs_check(var, x,s):
        return False
    s[var] = x
    return s

def occurs_check(var, x, s):
    # print("occure check",var, x, s)
    if var == x:
        return True
    if is_variable(x,variables) and x in s:
        return occurs_check(var, s[x], s)
    if isinstance(x, list):
        any(occurs_check(var, term, s) for term in x)
    return False

def substitute(s, expr, variables):
    if is_variable(expr, variables):
        return s.get(expr, expr)
    if isinstance(expr, list):
        return [substitute(s,exp, variables) for exp in expr]
    return expr


In [80]:
for c1, clause1 in list(clauses_map.items())[:22]:
    for c2, clause2 in list(clauses_map.items())[:10]:
        if is_negation(c1,c2, pred_map):
            for lit1 in clause1:
                for lit2 in clause2:
                    # if [clause1, clause2] in negated:
                    if lit1 != lit2:
                        # print("lit1","lit2", lit1, lit2, [clause1, clause2])
                        res = unify(lit1, lit2)
                        # print("Res",res)
                        if res is not False:
                            print("******************************")
                            print( "True",res, lit1, lit2, clause1, clause2, res)

******************************
True {c1: x, c2: x} [Q.Bat, c1] [Q.Bat, x] [[Q.Bat, c1], [Q.Elegant, c1], [Q.Brown, c2], [Q.HasBarrel, x], [Q.Bat_animal, x]] [[Q.Bat, x], [Q.Bat_animal, x], [Q.Bat_baseball, x]] {c1: x, c2: x}
******************************
True {c1: x, c2: x} [Q.Bat, c1] [Q.Bat, x] [[Q.CanFly, x], [Q.Bat, c1], [Q.Elegant, c1], [Q.Brown, c2], [Q.Bat_baseball, x]] [[Q.Bat, x], [Q.Bat_animal, x], [Q.Bat_baseball, x]] {c1: x, c2: x}
******************************
True {c1: x, c2: x} [Q.Bat, x] [Q.Bat, c1] [[Q.Bat, x], [Q.Bat_animal, x], [Q.UsedInBaseball, x]] [[Q.Bat, c1]] {c1: x, c2: x}


### Resolve and return resolvents and update predicate & clause maps respctively.

In [81]:
def can_resolve(clause1, clause2, pred_map):
    for lit1 in pred_map[clause1]:
        if isinstance(lit1, Not) or lit1.is_Not:
            pred = lit1.args[0].args[0] #first returns Not(this_lit), access this_lit
            if pred in clause2:
                return True
        else:
            pred = Not(lit1)
            if pred in clause2:
                return True
    return False

def resolve(clause1, clause2, pred_map, clauses_map):
    resolvents = set()
    for lit1, lit1_list in zip(clause1, clauses_map[clause1]):
        for lit2, lit2_list in zip(clause2, clauses_map[clause2]):
            # print(lit1, lit2, clause1, clause2)
            if is_complement(lit1, lit2):
                #unify
                # print("resolve")
                sub = unify(lit1_list,lit2_list)
                print("substitution",sub)
                if sub is not None:
                    for l in clause1:
                        if l != lit1:
                            print(substitute(sub,l, variables))
                    clause1_new = [substitute(sub,l, variables) for l in clause1 if l != lit1]
                    clause2_new = [substitute( sub, l,variables) for l in clause2 if l != lit2]
                    # print(clause1_new, clause2_new)
                    new_clause = set(clause1_new + clause2_new)
                    update_pred_map(tuple(new_clause), pred_map)
                    update_clauses_map(tuple(new_clause), clauses_map)
                    resolvents.add(tuple(new_clause))
    return resolvents

In [82]:
def resolution_refutation(clauses, pred_map, clauses_map):
    clauses = clauses_map.keys()
    allclauses = set(clauses)
    newclauses = set(clauses)
    while True:
        generated = set()
        clauses_list = [(c1, c2) for c1 in allclauses for c2 in newclauses if c1 != c2 and is_negation(c1, c2, pred_map)]
        # print(clauses_list)
        newclauses = set()
        results = []
        resolvents = None
        for c1, c2 in clauses_list:
            resolvents = resolve(c1, c2, pred_map, clauses_map)
            results.extend(resolvents)
            # print("*********************")
            # print(resolvents)
            # print(clauses)
            for res in resolvents:
                if res == ():
                    #empty clause
                    return True
                if res not in allclauses:
                    newclauses.add(res)
                    generated.add(res)
        if generated.issubset(allclauses):
            return False, resolvents, generated
        allclauses.update(newclauses)


### Resolution Refutation
For KB2, never terminates or there is a probability, it does, rarely.

In [84]:
resolution_refutation(clauses_map, pred_map, clauses_map)

KeyboardInterrupt: 

In [130]:
!sudo apt-get install texlive-xetex texlive-fonts-recommended
!sudo apt-get install texlive-xetex texlive-fonts-recommended texlive-plain-generic
!sudo apt-get install texlive
!pip install pandoc
!apt-get install pandoc

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following additional packages will be installed:
  dvisvgm fonts-droid-fallback fonts-lato fonts-lmodern fonts-noto-mono
  fonts-texgyre fonts-urw-base35 libapache-pom-java libcommons-logging-java
  libcommons-parent-java libfontbox-java libgs9 libgs9-common libidn12
  libijs-0.35 libjbig2dec0 libkpathsea6 libpdfbox-java libptexenc1 libruby3.0
  libsynctex2 libteckit0 libtexlua53 libtexluajit2 libwoff1 libzzip-0-13
  lmodern poppler-data preview-latex-style rake ruby ruby-net-telnet
  ruby-rubygems ruby-webrick ruby-xmlrpc ruby3.0 rubygems-integration t1utils
  teckit tex-common tex-gyre texlive-base texlive-binaries texlive-latex-base
  texlive-latex-extra texlive-latex-recommended texlive-pictures
  texlive-plain-generic tipa xfonts-encodings xfonts-utils
Suggested packages:
  fonts-noto fonts-freefont-otf | fonts-freefont-ttf libavalon-framework-java
  libcommons-logging-java-doc lib

In [131]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [132]:
!cp "/content/drive/MyDrive/Colab Notebooks/KB2-cs830-final-course-project-sushma-anand-akoju-updated.ipynb" /content/

In [133]:
%cd /content/
!jupyter nbconvert --to PDF "/content/KB2-cs830-final-course-project-sushma-anand-akoju-updated.ipynb"

/content
[NbConvertApp] Converting notebook /content/KB2-cs830-final-course-project-sushma-anand-akoju-updated.ipynb to PDF
[NbConvertApp] Writing 158927 bytes to notebook.tex
[NbConvertApp] Building PDF
[NbConvertApp] Running xelatex 3 times: ['xelatex', 'notebook.tex', '-quiet']
[NbConvertApp] Running bibtex 1 time: ['bibtex', 'notebook']
[NbConvertApp] WARNING | bibtex had problems, most likely because there were no citations
[NbConvertApp] PDF successfully created
[NbConvertApp] Writing 123542 bytes to /content/KB2-cs830-final-course-project-sushma-anand-akoju-updated.pdf
